In [1]:
from flask import Flask, render_template, jsonify
from flask_socketio import SocketIO, emit
import threading
import cv2
import mediapipe as mp
import time
import os
import csv
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from flask_cors import CORS

# Initialize Flask and Flask-SocketIO
app = Flask(__name__)

# Allow cross-origin requests from React app running on localhost:5173
socketio = SocketIO(app, cors_allowed_origins="http://localhost:5173")
key_face_landmarks_indices = [1, 33, 263, 61, 291, 199]  

# Camera and MediaPipe setup
mp_holistic = mp.solutions.holistic
holistic = mp_holistic.Holistic(static_image_mode=False, 
                                model_complexity=2, 
                                min_detection_confidence=0.4, 
                                min_tracking_confidence=0.4)

cap = None
is_running = False
landmark_data = []  # Store landmarks here

# Define model and label encoder
model = load_model('gesture_to_word_modelv2.0.h5')
labels = [
    'Bar', 'hajurama', 'baishak', 'pariwar', 'Dikka', 'bigyapan', 'mama', 
    'Eklopan', 'aama', 'ashar', 'bass', 'ashoj', 'shreemati', 'chora', 
    'buba', 'shreeman', 'Bhok', 'baba', 'Dukhi', 'Achamma', 'bhauju', 
    'angalo', 'udharo', 'Alchi', 'bhadra', 'kaki', 'chori', 'Sahanayak kosadhyaksa', 
    'hajurbuba', 'chaitra', 'bhena', 'maiju', 'kaka'
]
label_encoder = LabelEncoder()
label_encoder.fit(labels)



# Start the camera feed
@app.route('/start')
def start_camera():
    global is_running, landmark_data
    if not is_running:
        is_running = True
        landmark_data = []  # Reset the landmark data
        threading.Thread(target=process_camera_feed, daemon=True).start()
    return "Camera started"

# Stop the camera feed
@app.route('/stop')
def stop_camera():
    global is_running
    is_running = False
    # Save landmarks to CSV when stopped
    save_landmarks_to_csv(landmark_data, './camera_output_signals.csv')
    # Predict gesture using saved CSV
    predicted_word = predict_gesture('./camera_output_signals.csv')
    print(predicted_word)
    return jsonify({"predicted_word": predicted_word})

# Process camera feed and extract landmarks
def process_camera_feed():
    global cap, is_running, landmark_data
    cap = cv2.VideoCapture(0)

    while is_running:
        ret, frame = cap.read()
        if not ret:
            break

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic.process(rgb_frame)

        frame_landmarks = {}
        if results.pose_landmarks:
            frame_landmarks['pose'] = [(landmark.x, landmark.y, landmark.z) for landmark in results.pose_landmarks.landmark]
        if results.face_landmarks:
            frame_landmarks['face'] = [(landmark.x, landmark.y, landmark.z) for idx, landmark in enumerate(results.face_landmarks.landmark) if idx in key_face_landmarks_indices]
        if results.left_hand_landmarks:
            frame_landmarks['left_hand'] = [(landmark.x, landmark.y, landmark.z) for landmark in results.left_hand_landmarks.landmark]
        if results.right_hand_landmarks:
            frame_landmarks['right_hand'] = [(landmark.x, landmark.y, landmark.z) for landmark in results.right_hand_landmarks.landmark]

        landmark_data.append(frame_landmarks)
        
        
        # Draw landmarks on the frame
        frame_with_landmarks = draw_landmarks(frame, results)


        # Encode the frame as JPEG
        _, jpeg = cv2.imencode('.jpg', frame_with_landmarks)
        frame_bytes = jpeg.tobytes()

        # Emit the frame to the frontend
        socketio.emit('video_frame', frame_bytes)
        time.sleep(0.03)  # Simulate FPS

    cap.release()
    
    

    
    
# Function to draw landmarks on the frame
def draw_landmarks(frame, results):
    if results.pose_landmarks:
        mp.solutions.drawing_utils.draw_landmarks(frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    
    if results.face_landmarks:
        mp.solutions.drawing_utils.draw_landmarks(frame, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS)
    
    if results.left_hand_landmarks:
        mp.solutions.drawing_utils.draw_landmarks(frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    
    if results.right_hand_landmarks:
        mp.solutions.drawing_utils.draw_landmarks(frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    
    return frame    

# Save landmarks to CSV
def save_landmarks_to_csv(landmark_data, csv_path):
    with open(csv_path, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        header = ['frame', 'type', 'index', 'x', 'y', 'z']
        csvwriter.writerow(header)

        for frame_index, frame_data in enumerate(landmark_data):
            for landmark_type, landmarks in frame_data.items():
                for idx, (x, y, z) in enumerate(landmarks):
                    csvwriter.writerow([frame_index, landmark_type, idx, x, y, z])

# Prepare input data for gesture prediction
def prepare_input_data(csv_file_path, sequence_length=30):
    df = pd.read_csv(csv_file_path)
    landmarks = df[['x', 'y', 'z']].values
    num_frames = len(landmarks)

    sequences = []
    for start in range(0, num_frames - sequence_length, sequence_length):
        sequence = landmarks[start:start + sequence_length]
        sequences.append(sequence)

    return np.array(sequences)

# Predict gesture based on CSV file
def predict_gesture(csv_file_path, sequence_length=30):
    sequences = prepare_input_data(csv_file_path, sequence_length)
    predictions = model.predict(sequences)
    predicted_class_idx = np.argmax(predictions, axis=1)
    predicted_labels = label_encoder.inverse_transform(predicted_class_idx)
    
    # Get the most frequent prediction
    label_counts = Counter(predicted_labels)
    most_common_label, most_common_count = label_counts.most_common(1)[0]
    
    return most_common_label




@socketio.on('connect')
def handle_connect():
    print("Client connected")

@socketio.on('disconnect')
def handle_disconnect():
    print("Client disconnected")

if __name__ == '__main__':
    CORS(app, origins=["http://localhost:5173"])
    socketio.run(app, host='0.0.0.0', port=5000, allow_unsafe_werkzeug=True)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.3:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [18/Nov/2024 06:20:37] "GET /socket.io/?EIO=4&transport=polling&t=ao6e8wc4 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Nov/2024 06:20:37] "POST /socket.io/?EIO=4&transport=polling&t=ao6n5zly&sid=vSb1LX5idY0ts-UbAAAA HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Nov/2024 06:20:37] "GET /socket.io/?EIO=4&transport=polling&t=ao6n6n1i&sid=vSb1LX5idY0ts-UbAAAA HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Nov/2024 06:20:37] "GET /socket.io/?EIO=4&transport=polling&t=ao6vxysb&sid=vSb1LX5idY0ts-UbAAAA HTTP/1.1" 200 -


Client connected


INFO:werkzeug:127.0.0.1 - - [18/Nov/2024 06:20:38] "GET /start HTTP/1.1" 200 -
E:\AnacondaS\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
E:\AnacondaS\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 229ms/step


...

Client connected


INFO:werkzeug:127.0.0.1 - - [18/Nov/2024 06:52:10] "GET /start HTTP/1.1" 200 -


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 


E:\AnacondaS\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
INFO:werkzeug:127.0.0.1 - - [18/Nov/2024 06:52:31] "GET /stop HTTP/1.1" 200 -


chora
